In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns',False)

In [3]:
url="https://raw.githubusercontent.com/WalePhenomenon/climate_change/master/fuel_ferc1.csv"
fuel_quality_frc= pd.read_csv(url,error_bad_lines=False)

In [4]:
fuel_quality_frc.head(5)

,record_id,utility_id_ferc1,report_year,plant_name_ferc1,fuel_type_code_pudl,fuel_unit,fuel_qty_burned,fuel_mmbtu_per_unit,fuel_cost_per_unit_burned,fuel_cost_per_unit_delivered,fuel_cost_per_mmbtu
0,f1_fuel_1994_12_1_0_7,1,1994,rockport,coal,ton,5377489.0,16.590,18.59,18.53,1.121
1,f1_fuel_1994_12_1_0_10,1,1994,rockport total plant,coal,ton,10486945.0,16.592,18.58,18.53,1.120
2,f1_fuel_1994_12_2_0_1,2,1994,gorgas,coal,ton,2978683.0,24.130,39.72,38.12,1.650
3,f1_fuel_1994_12_2_0_7,2,1994,barry,coal,ton,3739484.0,23.950,47.21,45.99,1.970
4,f1_fuel_1994_12_2_0_10,2,1994,chickasaw,gas,mcf,40533.0,1.000,2.77,2.77,2.570


In [5]:
fuel_quality_frc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29523 entries, 0 to 29522
Data columns (total 11 columns):
record_id                       29523 non-null object
utility_id_ferc1                29523 non-null int64
report_year                     29523 non-null int64
plant_name_ferc1                29523 non-null object
fuel_type_code_pudl             29523 non-null object
fuel_unit                       29343 non-null object
fuel_qty_burned                 29523 non-null float64
fuel_mmbtu_per_unit             29523 non-null float64
fuel_cost_per_unit_burned       29523 non-null float64
fuel_cost_per_unit_delivered    29523 non-null float64
fuel_cost_per_mmbtu             29523 non-null float64
dtypes: float64(5), int64(2), object(4)
memory usage: 2.5+ MB


In [6]:
fuel_quality_frc.describe()

,utility_id_ferc1,report_year,fuel_qty_burned,fuel_mmbtu_per_unit,fuel_cost_per_unit_burned,fuel_cost_per_unit_delivered,fuel_cost_per_mmbtu
count,29523.000000,29523.000000,2.952300e+04,29523.000000,29523.000000,2.952300e+04,29523.000000
mean,118.601836,2005.806050,2.622119e+06,8.492111,208.649031,9.175704e+02,19.304354
std,74.178353,7.025483,9.118004e+06,10.600220,2854.490090,6.877593e+04,2091.540939
min,1.000000,1994.000000,1.000000e+00,0.000001,-276.080000,-8.749370e+02,-41.501000
25%,55.000000,2000.000000,1.381700e+04,1.024000,5.207000,3.778500e+00,1.940000
50%,122.000000,2006.000000,2.533220e+05,5.762694,26.000000,1.737100e+01,4.127000
75%,176.000000,2012.000000,1.424034e+06,17.006000,47.113000,4.213700e+01,7.745000
max,514.000000,2018.000000,5.558942e+08,341.260000,139358.000000,7.964521e+06,359278.000000


In [14]:
lst_avg_fuel_cost= fuel_quality_frc.groupby('fuel_type_code_pudl')['fuel_cost_per_unit_burned'].mean().nsmallest(1)
lst_avg_fuel_cost

fuel_type_code_pudl
gas    13.659397
Name: fuel_cost_per_unit_burned, dtype: float64

In [15]:
std_dev,sev_fifth_per= fuel_quality_frc['fuel_mmbtu_per_unit'].std(), fuel_quality_frc['fuel_mmbtu_per_unit'].quantile(0.75)

In [16]:
print('standard deviation: {}\n75th percentile: {}'.format(np.round(std_dev,2),np.round(sev_fifth_per,2)))

standard deviation: 10.6
75th percentile: 17.01


In [17]:
f_qty_skewness,f_qty_kurt=fuel_quality_frc['fuel_qty_burned'].skew(),fuel_quality_frc['fuel_qty_burned'].kurt()

In [18]:
print('Skewness of Fuel Quantity Burned: {}\nKurtosis of Fuel Quantity Burned: {}'.format(np.round(f_qty_skewness,2),np.round(f_qty_kurt,2)))

Skewness of Fuel Quantity Burned: 15.85
Kurtosis of Fuel Quantity Burned: 651.37


In [19]:
fuel_quality_frc.isnull().sum()

record_id                         0
utility_id_ferc1                  0
report_year                       0
plant_name_ferc1                  0
fuel_type_code_pudl               0
fuel_unit                       180
fuel_qty_burned                   0
fuel_mmbtu_per_unit               0
fuel_cost_per_unit_burned         0
fuel_cost_per_unit_delivered      0
fuel_cost_per_mmbtu               0
dtype: int64

In [20]:
np.round(fuel_quality_frc['fuel_unit'].isnull().value_counts(normalize=True),3)*100

False    99.4
True      0.6
Name: fuel_unit, dtype: float64

In [27]:
fuel_quality_frc.corr()['fuel_cost_per_unit_burned'].sort_values()

utility_id_ferc1               -0.037863
fuel_qty_burned                -0.018535
fuel_mmbtu_per_unit            -0.010034
fuel_cost_per_mmbtu            -0.000437
fuel_cost_per_unit_delivered    0.011007
report_year                     0.013599
fuel_cost_per_unit_burned       1.000000
Name: fuel_cost_per_unit_burned, dtype: float64

In [78]:
new_df=fuel_quality_frc[fuel_quality_frc['report_year'].isin([1994,1998])]
new_df=new_df[['report_year','fuel_type_code_pudl','fuel_cost_per_unit_burned']]
new_df=pd.DataFrame(new_df.groupby(['report_year','fuel_type_code_pudl'])['fuel_cost_per_unit_burned'].sum().reset_index())
new_df.head(10)

,report_year,fuel_type_code_pudl,fuel_cost_per_unit_burned
0,1994,coal,14984.572
1,1994,gas,10792.496
2,1994,nuclear,227983.354
3,1994,oil,9362.194
4,1994,other,116.737
5,1994,waste,52.762
6,1998,coal,11902.597
7,1998,gas,7475.596
8,1998,nuclear,117788.269
9,1998,oil,2608.657


In [86]:
percent_change=np.round((new_df.iloc[6,2]-new_df.iloc[0,2])/new_df.iloc[0,2],2)*100

In [87]:
print('Percentage Change between 1994 compared to 1998: {}'.format(percent_change))

Percentage Change between 1994 compared to 1998: -21.0


In [97]:
largest_year=fuel_quality_frc.groupby(['report_year'])['fuel_cost_per_unit_delivered'].mean().sort_values().nlargest(1)

In [95]:
print('Year with highest fuel cost per unit: {}'.format(largest_year))

Year with highest fuel cost per unit: report_year
1997    11140.197239
Name: fuel_cost_per_unit_delivered, dtype: float64
